**Содержание**<a id='toc0_'></a>    
- [Парсинг LinkedIn](#toc1_1_1_)    
      - [Основные этапы](#toc1_1_1_1_)    
      - [Основные результаты](#toc1_1_1_2_)    
- [Обработка текстов](#toc1_1_2_)    
      - [Основные этапы](#toc1_1_2_1_)    
      - [Основные результаты](#toc1_1_2_2_)    
- [Построение языковой модели](#toc1_1_3_)    
      - [Основные этапы](#toc1_1_3_1_)    
      - [Основные результаты](#toc1_1_3_2_)    
      - [Топ-10 тем по ключевым словам (рус)](#toc1_1_3_3_)    
      - [Модель на англоязчных текстах](#toc1_1_3_4_)    
      - [Топ-10 тем по ключевым словам (англ)](#toc1_1_3_5_)    
- [Подготовка данных для языковой модели по самым популярным постам](#toc1_1_4_)    
- [Построение языковой модели по топ-50 популярным постам](#toc1_1_5_)    
      - [Топ-5 популярных тем по реакциям (рус)](#toc1_1_5_1_)    
      - [Топ-5 популярных тем по реакциям (англ)](#toc1_1_5_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import pandas as pd
import time
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import pycld2 as cld2

from pymystem3 import Mystem

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import nltk 
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import gensim
import gensim.corpora as corpora

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anshilina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anshilina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/anshilina/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### <a id='toc1_1_1_'></a>[Парсинг LinkedIn](#toc0_)

#### <a id='toc1_1_1_1_'></a>[Основные этапы](#toc0_)
- Создание кода для сбора данных
    1. Встроить прокси
    2. Добавить инструменты для сохранения данных
- Сбор данных
- Предобработка данных
    1. Убрать эмодзи
    2. Привести слова в начальную форму
    3. Убрать лишние текстовые символы
    4. Убрать стоп-слова и слова/паразиты
    5. Сохранить в таблицу

#### <a id='toc1_1_1_2_'></a>[Основные результаты](#toc0_)
- Был модифицирован изначальный код для парсинга данных: в конфигурацию были добавлены прокси, в функции был добавлен код для сохранения данны
- Была собрана информация о **288** пользователях LinkedIn, которые были отобраны по ключевым словам: *data science mentors*, *mentors in it*, *teach it*, *наставники data science* и проч. Мы собирали следующую информацию: ФИО, место работы, ссылка на профиль, публикации и количество реакций к ним
- Предпочтение отдавалось пользователям, которые ведут аккаунты не только на английском, но и на русском языках. Также в поисковике LinkedIn был выставлен фильтр на локацию: Россия и страны СНГ. Среди **288** пользователей 84 человек публиковали посты на английском, 32 – на русском, и еще 10 на азербайджанском. Остальные не публиковали посты
- Все публикации пользователей были загружены в формате вложенных списков со структурой *[['ТЕКСТ ПУБЛИКАЦИИ', 'КОЛИЧЕСТВО РЕАКЦИЙ'],['ТЕКСТ ПУБЛИКАЦИИ', 'КОЛИЧЕСТВО РЕАКЦИЙ'] и т.д.]*. Такая структура может быть легко преобразована как в отдельные корпуса текстов, реакций к ним, так и в один единый текст от каждого автора. Продробнее об этом на этапе предобработки данных
- Сбор информации проводился до тех пор пока не были заблокированы все созданные для достижения целей проекта аккаунты
- Сырые данные были сохранены в массиве *russian_mentors_raw.csv*

In [94]:
USER_LOGIN = 'EMAIL'
USER_PASSWORD = 'PASSWORD'

# To collect data and convert to df
df = {'name':[], 'works_at':[], 'profile_url': [], 'posts': []}

In [95]:
def get_and_print_user_posts(driver, posts_url):
    driver.get(posts_url)

    # Simulate scrolling to capture all posts
    SCROLL_PAUSE_TIME = 3

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    # We can adjust this number to get more posts
    NUM_SCROLLS = 5

    for i in range(NUM_SCROLLS):
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Parsing posts
    src = driver.page_source

    # Now using beautiful soup
    soup = BeautifulSoup(src, "html.parser")
    # soup.prettify()
    if soup is None:
        return 
    
    posts = soup.find_all('li', class_='profile-creator-shared-feed-update__container')
    # print(posts)

    print(f'Number of posts: {len(posts)}')
    posts_list = []
    for post_src in posts:
        post_text_div = post_src.find('div', {'class': 'feed-shared-update-v2__description-wrapper mr2'})

        if post_text_div is not None:
            post_text = post_text_div.find('span', {'dir': 'ltr'})
        else:
            post_text = None

        # If post text is found
        if post_text is not None:
            post_text = post_text.get_text().strip()
            print(f'Post text: {post_text}')

        reaction_cnt = post_src.find('span', {'class': 'social-details-social-counts__reactions-count'})

        # If number of reactions is written as text
        # It has different class name
        if reaction_cnt is None:
            reaction_cnt = post_src.find('span', {'class': 'social-details-social-counts__social-proof-text'})

        if reaction_cnt is not None:
            reaction_cnt = reaction_cnt.get_text().strip()
            print(f'Reactions: {reaction_cnt}')
        
    # Add to a dict
        if post_text is not None:
            posts_list.append([post_text, reaction_cnt])
    df['posts'].append(posts_list)

    return

In [96]:
def get_and_print_profile_info(driver, profile_url):
    # Open the link
    driver.get(profile_url)

    # Extracting data from page with BeautifulSoup
    src = driver.page_source

    soup = BeautifulSoup(src, "html.parser")

    # Extracting the HTML of the complete introduction box
    # that contains the name, company name, and the location
    if soup is None:
        return
    
    intro = soup.find('div', {'class': 'pv-text-details__left-panel'})

    # print(intro)

    # In case of an error, try changing the tags used here.
    name_loc = intro.find("h1")

    # Extracting the Name
    name = name_loc.get_text().strip()
    # strip() is used to remove any extra blank spaces

    works_at_loc = intro.find("div", {'class': 'text-body-medium'})

    # this gives us the HTML of the tag in which the Company Name is present
    # Extracting the Company Name
    works_at = works_at_loc.get_text().strip()

    print("Name -->",  name,
          "\nWorks At -->", works_at)

    POSTS_URL_SUFFIX = 'recent-activity/all/'

    time.sleep(2)

    # Get current url from browser
    cur_profile_url = driver.current_url
    print(cur_profile_url)
    
    # Add to a Dict
    df['name'].append(name)
    df['works_at'].append(works_at)
    df['profile_url'].append(cur_profile_url)

    # Parse posts
    get_and_print_user_posts(driver, cur_profile_url + POSTS_URL_SUFFIX)

In [ ]:
if __name__ == '__main__':
    # start Chrome browser
    caps = DesiredCapabilities().CHROME

    caps['pageLoadStrategy'] = 'eager'

    # NOTE: If you need VPN use it: 
    # chrome_options = webdriver.ChromeOptions()
    # PROXY = "147.135.54.182:3128"
    # chrome_options.add_argument('--proxy-server=%s' % PROXY)
    # driver = webdriver.Chrome(chrome_options=chrome_options)
    
    driver = webdriver.Chrome() # can add specific driver

    # Opening linkedIn's login page
    # NOTE: We need to turn of 2 step authentification
    driver.get("https://linkedin.com/uas/login")

    # waiting for the page to load
    time.sleep(5)

    # entering username
    username = driver.find_element(By.ID, "username")

    # In case of an error, try changing the element
    # tag used here.

    # Enter Your Email Address
    username.send_keys(USER_LOGIN)

    # entering password
    pword = driver.find_element(By.ID, "password")
    # In case of an error, try changing the element
    # tag used here.

    # Enter Your Password
    pword.send_keys(USER_PASSWORD)

    # Clicking on the log in button
    # Format (syntax) of writing XPath -->
    # //tagname[@attribute='value']
    driver.find_element(By.XPATH, "//button[@type='submit']").click()
    time.sleep(60)

    # Open search page
    # driver.get('https://www.linkedin.com/search/results/people/?geoUrn=%5B%22101728296%22%5D&keywords=data%20science%20mentor&origin=GLOBAL_SEARCH_HEADER&sid=MzX')
    driver.get('https://www.linkedin.com/search/results/people/?geoUrn=%5B%22101728296%22%5D&keywords=data%20science%20mentor&origin=FACETED_SEARCH&profileLanguage=%5B%22en%22%2C%22ru%22%5D&sid=BWL')
    
    profile_urls = []

    NUM_PAGES_TO_PARSE = 20

    # Iterate over pages of search results
    # to collect profile urls
    for i in range(NUM_PAGES_TO_PARSE):
        search_result_links = driver.find_elements(By.CSS_SELECTOR, "div.entity-result__item a.app-aware-link")

        for link in search_result_links:
            href = link.get_attribute("href")
            if 'linkedin.com/in' in href:
                profile_urls.append(href)

        next_button = driver.find_element(By.CLASS_NAME,'artdeco-pagination__button--next')
        next_button.click()
        time.sleep(2)


    profile_urls = list(set(profile_urls))

    print(profile_urls)

    # Parse profile urls
    for profile_url in profile_urls:
        get_and_print_profile_info(driver, profile_url)
        time.sleep(5)

    # close the Chrome browser
    driver.quit()

Далее полученные данные сохраняются в формате csv. Корпус текстов и их авторы доступны в файле **russian_mentors_raw.csv**.

In [ ]:
# data = pd.DataFrame(df)
# data.to_csv('russian_mentors_raw.csv')

### <a id='toc1_1_2_'></a>[Обработка текстов](#toc0_)

#### <a id='toc1_1_2_1_'></a>[Основные этапы](#toc0_)
- Убрать эмодзи
- Привести слова в начальную форму
- Убрать лишние текстовые символы
- Убрать стоп-слова и слова-паразиты
- Сохранить в таблицу
- Совместить датасет с данными с информацией на основной странице

#### <a id='toc1_1_2_2_'></a>[Основные результаты](#toc0_)
- С помощью библиотеки *pycld2* был определен язык публикаций: русский и английский, только русский, только английский, азербайджанский (не включали в анализ)
- Из выборки были удалены те, кто указывал в качестве своей профессии маркетинг и менеджмент (согласно требованиям заказчика)
- Были удалены дубликаты, которые появились из-за того, что 22 человека попали в результаты поисков по разным вариациям тэгов и ключевых слов
- На данном этапе все публикации человека рассматриваются как один текст
- Рускоязычные и англоязычные посты предобрабатывались отдельно, но этапы для обоих языков схожие: приведение текстов к нижнему регистру, удаление всех символов и эмоджи и сохранение только текстовой информации (с помощью регулярных выражений, библиотека *re*), лемматизация токенов (библиотеки *pymystem3* и *nltk*), фильтрация токенов: удаление стоп-слов и слов-паразитов (библиотека *nltk*)
- На заключительном этапе мы получили отдельные слова текстов
- Данные были сохранены в таблицы *data_russian.csv* и *data_english.csv*

In [39]:
def detect_lang(series):
    '''
    Function to detect language of the text
    '''
    text_content = str(series)
    _, _, _, detected_language = cld2.detect(text_content,  returnVectors=True)
    for i in range(len(detected_language)):
        if ('RUSSIAN' in detected_language[i] and 
            ('Unknown' in detected_language[i] 
             or 'ENGLISH' in detected_language[i])):
            return 'RUSSIAN AND ENGLISH'
        if 'RUSSIAN' in detected_language[i]:
            return 'RUSSIAN'
        elif 'AZERBAIJANI' in detected_language[i]:
            return 'AZERBAIJANI'
        elif ('Unknown' in detected_language[i] or 'ENGLISH' in detected_language[i]):
            return 'ENGLISH'

In [40]:
def preprocess_rus_texts(text):
    '''
    Function to preprocess russian texts
    '''
    text = text.lower()

    text = re.sub(r'[^а-яё ]', ' ', text)
    text = " ".join(text.split())

    m = Mystem()
    text = m.lemmatize(text)

    russian_stopwords = stopwords.words('russian')
    russian_stopwords.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на', 'мой', 'наш', 'твой', 'ваш', 'свой',
                              'год', 'очень', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь', 'январь', 
                              'февраль', 'март', 'апрель', 'май', 'или', 'но', 'дабы', 'затем', 'потом', 'лишь', 'только', 'твой', 
                              'какой', 'который', 'такой', 'кто', 'все', 'тот', 'себя', 'меня', 'вас', 'он', 'мы', 'его', 'вы', 'вам', 
                              'ее', 'их', 'они', 'я', 'весь', 'мне', 'меня', 'таким', 'для', 'на', 'по', 'со', 'из', 'от', 'до', 'без', 
                              'над', 'под', 'за', 'при', 'после', 'во', 'не', 'же', 'то', 'бы', 'всего', 'итого', 'даже', 'да', 'нет', 
                              'ой', 'ого', 'эх', 'браво', 'здравствуйте', 'здравствуй', 'здравствовать', 'спасибо', 'извините', 'привет', 'один', 'два', 
                              'три', 'первый', 'второй', 'третий', 'скажем', 'может', 'допустим', 'честно говоря', 'например', 'на самом деле', 
                              'однако', 'вообще', 'в общем', 'вероятно', 'что-то', 'какой-то', 'где-то', 'как-то', 'дальше', 'ближе', 
                              'раньше', 'позже', 'когда-то', 'очень', 'часто', 'минимально', 'максимально', 'абсолютно', 'огромный', 'предельно', 
                              'сильно', 'слабо', 'самый', 'давать', 'всегда', 'однако'])
    filtered_text = " ".join([w for w in text if len(w) > 2 if w not in russian_stopwords])

    return filtered_text

In [41]:
def preprocess_eng_texts(text):
    '''
    Function to preprocess english texts
    '''
    text = text.lower()

    text = re.sub(r'[^a-z ]', ' ', text).split()

    stemmer = PorterStemmer() 
    lemmatizer = WordNetLemmatizer()
    stem_words=[stemmer.stem(w) for w in text]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]

    english_stopwords = stopwords.words('english')
    english_stopwords.extend(['http', 'lnkd'])
    filtered_text = " ".join([w for w in lemma_words if len(w) > 2 if w not in english_stopwords])

    return filtered_text

In [10]:
data = pd.read_csv('russian_mentors_raw.csv')
data.head(10)

,name,works_at,profile_url,posts
0,Елена Хасанова,"QA Head, QA Team-lead, QA-Lead, Наставник Янде...",https://www.linkedin.com/in/yelena-khassanova/,"[['Минск, кто ищет стажировки QA - для вас 🙂 '..."
1,Алексей Становкин,Ведущий Software QA Engineer,https://www.linkedin.com/in/alex7cri/,"[['https://lnkd.in/gx-UipHF#нетУбийствам'], ['..."
2,Дмитрий Липин,СТО в Yandex Platform Engineering,https://www.linkedin.com/in/dmitrylipin/,[['Очередной митап команды Yandex Infrastructu...
3,Ramil Gataullin,Data Engineer at Toptal,https://www.linkedin.com/in/yaugear/,[]
4,Александр Романов,CTO (CIO) at iRidi | Создаю и трансформирую си...,https://www.linkedin.com/in/romanovalexander/,[['Два дня преподавания в МГУ )))Провел 120 ст...
5,Aleksandr Shabanov,ML engineer,https://www.linkedin.com/in/alshabanov/,[['Hi everyone - I am looking for a new role a...
6,Pavel Kiryanov,QA Lead,https://www.linkedin.com/in/pavel-kiryanov-974...,"[['Я вот человек простой, чет написал и коммен..."
7,Maxim T.,DevOps Manager,https://www.linkedin.com/in/maxim-toropov/,"[[""I just earned a skill badge for MongoDB! Wh..."
8,Amir Safiullin,Machine Learning Engineer,https://www.linkedin.com/in/safiullinamir/,"[['22 июня, в 19:00, приглашаем на VK Tech Tal..."
9,Soslan Tabuev,datascience.xyz,https://www.linkedin.com/in/soslan-tabuev/,[['An interesting review of Transformer Zoo by...


In [9]:
data.shape

(303, 4)

По просьбе заказчика мы исключаем менеджеров и маркетологов из выборки.

In [44]:
# exclude managers and marketologists
data = data[(data['works_at'].str.contains('Manager')==False)
             & (data['works_at'].str.contains('manager')==False)
             & (data['works_at'].str.contains('Marketing')==False)
             & (data['works_at'].str.contains('marketing')==False)].copy()

In [45]:
print(f'Число дубликатов {data.duplicated().sum()}')
data.drop_duplicates(inplace=True)

Число дубликатов 29


In [46]:
data['language'] = data['posts'].apply(detect_lang)

data_russian = data.loc[(data['language'] == 'RUSSIAN') | (data['language'] == 'RUSSIAN AND ENGLISH')]
data_russian = data_russian.reset_index(drop=True)

data_english = data.loc[(data['language'] == 'ENGLISH') | (data['language'] == 'RUSSIAN AND ENGLISH')]
data_english = data_english.reset_index(drop=True)

In [47]:
data['language'].value_counts()

ENGLISH        84
RUSSIAN        32
AZERBAIJANI    10
Name: language, dtype: int64

Предобработаем русскоязычные и англоязычные тексты отдельно.

In [48]:
data_russian['posts_processed'] = data_russian['posts'].apply(preprocess_rus_texts)
data_russian.head()

,name,works_at,profile_url,posts,language,posts_processed
0,Елена Хасанова,"QA Head, QA Team-lead, QA-Lead, Наставник Янде...",https://www.linkedin.com/in/yelena-khassanova/,"[['Минск, кто ищет стажировки QA - для вас 🙂 '...",RUSSIAN,минск искать стажировка созревать поделиться п...
1,Александр Романов,CTO (CIO) at iRidi | Создаю и трансформирую си...,https://www.linkedin.com/in/romanovalexander/,[['Два дня преподавания в МГУ )))Провел 120 ст...,RUSSIAN,день преподавание мгу проводить студент получе...
2,Pavel Kiryanov,QA Lead,https://www.linkedin.com/in/pavel-kiryanov-974...,"[['Я вот человек простой, чет написал и коммен...",RUSSIAN,человек простой чета написать коммент читаюно ...
3,Александр Шобухов,CrowdFunding - Народное финансирование.,https://www.linkedin.com/in/%D0%B0%D0%BB%D0%B5...,[['Здравствуй Друг! Посмотри новый проект! Вы ...,RUSSIAN,друг посмотреть новый проект человек предприим...
4,Aleksandr Romanov,CTO (CIO) at iRidium mobile | I Create and Tra...,https://www.linkedin.com/in/romanovalexander/,[['Два дня преподавания в МГУ )))Провел 120 ст...,RUSSIAN,день преподавание мгу проводить студент получе...


In [49]:
data_english['posts_processed'] = data_english['posts'].apply(preprocess_eng_texts)
data_english.head()

,name,works_at,profile_url,posts,language,posts_processed
0,Алексей Становкин,Ведущий Software QA Engineer,https://www.linkedin.com/in/alex7cri/,"[['https://lnkd.in/gx-UipHF#нетУбийствам'], ['...",ENGLISH,uiphf engin interview without look resum crede...
1,Дмитрий Липин,СТО в Yandex Platform Engineering,https://www.linkedin.com/in/dmitrylipin/,[['Очередной митап команды Yandex Infrastructu...,ENGLISH,yandex infrastructur backend devop sre open so...
2,Aleksandr Shabanov,ML engineer,https://www.linkedin.com/in/alshabanov/,[['Hi everyone - I am looking for a new role a...,ENGLISH,everyon look new role would appreci support th...
3,Amir Safiullin,Machine Learning Engineer,https://www.linkedin.com/in/safiullinamir/,"[['22 июня, в 19:00, приглашаем на VK Tech Tal...",ENGLISH,tech talk coheua implicit feedback need vktech...
4,Soslan Tabuev,datascience.xyz,https://www.linkedin.com/in/soslan-tabuev/,[['An interesting review of Transformer Zoo by...,ENGLISH,interest review transform zoo grigori sapunov ...


In [50]:
print(f'Кол-во русскоязычных авторов: {data_russian.shape[0]}')

Кол-во русскоязычных авторов: 32


In [51]:
print(f'Кол-во англоязычных авторов:{data_english.shape[0]}')

Кол-во англоязычных авторов:84


Остальные пользователи, которые попали в выборку, еще не опубликовали публикации на своих страницах.

In [52]:
data_russian['words'] = data_russian['posts_processed'].apply(lambda series: series.split())
data_russian['words'].head()

0    [минск, искать, стажировка, созревать, поделит...
1    [день, преподавание, мгу, проводить, студент, ...
2    [человек, простой, чета, написать, коммент, чи...
3    [друг, посмотреть, новый, проект, человек, пре...
4    [день, преподавание, мгу, проводить, студент, ...
Name: words, dtype: object

In [53]:
data_english['words'] = data_english['posts_processed'].apply(lambda series: series.split())
data_english['words'].head()

0    [uiphf, engin, interview, without, look, resum...
1    [yandex, infrastructur, backend, devop, sre, o...
2    [everyon, look, new, role, would, appreci, sup...
3    [tech, talk, coheua, implicit, feedback, need,...
4    [interest, review, transform, zoo, grigori, sa...
Name: words, dtype: object

Данные сохраняем в отдельные массивы **data_russian.csv** и **data_english.csv**.

In [54]:
data_russian.to_csv('data_russian.csv')
data_english.to_csv('data_english.csv')

### <a id='toc1_1_3_'></a>[Построение языковой модели](#toc0_)

#### <a id='toc1_1_3_1_'></a>[Основные этапы](#toc0_)
- Анализ и выделение 100 постов с большей частотой встречающихся слов
- Создание Topic Modeling на всех постах
- Создание тетрадки в Jupiter

#### <a id='toc1_1_3_2_'></a>[Основные результаты](#toc0_)
- Были выделены 100 постов с большей частотой встречающихся слов
- Основной фокус исследования на русскоязычных постах (в соответсвии с запросом заказчика)
- Но в качестве эксперимента мы также строим англоязычную языковую модель (в отдельном разделе). Это решение связано с тем, что многие менторы ориентируются на более широкую аудиторию, не только русскоязычную
- Для построения языковой модели мы рассматривали каждый пост автора – это отдельный текст (удалось выделить топ-10 тем)
- Была построена языковая модель по топ-10 популярным темам (по количеству реакций), которые обозревали менторы
- Результаты моделирования были проинтерпретированы: были расшифрованы полученные темы
- Код для сбора, обработки данных и для построения модели был структурирован в Jupiter тетрадке

In [55]:
data.reset_index(drop=True, inplace=True)

In [56]:
data.head(10)

,name,works_at,profile_url,posts,language
0,Елена Хасанова,"QA Head, QA Team-lead, QA-Lead, Наставник Янде...",https://www.linkedin.com/in/yelena-khassanova/,"[['Минск, кто ищет стажировки QA - для вас 🙂 '...",RUSSIAN
1,Алексей Становкин,Ведущий Software QA Engineer,https://www.linkedin.com/in/alex7cri/,"[['https://lnkd.in/gx-UipHF#нетУбийствам'], ['...",ENGLISH
2,Дмитрий Липин,СТО в Yandex Platform Engineering,https://www.linkedin.com/in/dmitrylipin/,[['Очередной митап команды Yandex Infrastructu...,ENGLISH
3,Ramil Gataullin,Data Engineer at Toptal,https://www.linkedin.com/in/yaugear/,[],None
4,Александр Романов,CTO (CIO) at iRidi | Создаю и трансформирую си...,https://www.linkedin.com/in/romanovalexander/,[['Два дня преподавания в МГУ )))Провел 120 ст...,RUSSIAN
5,Aleksandr Shabanov,ML engineer,https://www.linkedin.com/in/alshabanov/,[['Hi everyone - I am looking for a new role a...,ENGLISH
6,Pavel Kiryanov,QA Lead,https://www.linkedin.com/in/pavel-kiryanov-974...,"[['Я вот человек простой, чет написал и коммен...",RUSSIAN
7,Amir Safiullin,Machine Learning Engineer,https://www.linkedin.com/in/safiullinamir/,"[['22 июня, в 19:00, приглашаем на VK Tech Tal...",ENGLISH
8,Soslan Tabuev,datascience.xyz,https://www.linkedin.com/in/soslan-tabuev/,[['An interesting review of Transformer Zoo by...,ENGLISH
9,Alexander Konkin,Business Data Science Team Lead – Exness,https://www.linkedin.com/in/alexander-konkin-0...,[],None


In [57]:
data.loc[(data['language'] == 'RUSSIAN') | (data['language'] == 'RUSSIAN AND ENGLISH'), 'posts']

0      [['Минск, кто ищет стажировки QA - для вас 🙂 '...
4      [['Два дня преподавания в МГУ )))Провел 120 ст...
6      [['Я вот человек простой, чет написал и коммен...
17     [['Здравствуй Друг! Посмотри новый проект! Вы ...
19     [['Два дня преподавания в МГУ )))Провел 120 ст...
22     [['Решил написать ответ на часто задаваемый во...
26     [["26 сентября в рамках международной онлайн-к...
37     [['К сожалению, бизнес-модель многих компаний ...
40     [['Друзья, я с новостями!⠀Студия творческих пр...
42     [['Свежее и очень личное, дистиллированное до ...
45     [['Друзья! В нашу чудесную команду SuperJob ра...
58     [['Если вы новичок в интернет бизнесе и только...
61     [['С удовольствием сообщаю, что я начинаю рабо...
62     [['Хочется сменить финансы в строительной комп...
70     [['Команда номер 1 :-)) Спасибо вам за возможн...
76     [['С удовольствием сообщаю, что я начинаю рабо...
79     [['Лайкните, пожалуйста, моё видео «Борис Викт...
81     [['Присоединяйтесь к ФИН

Теперь посмотрим, сколько в сумме русскоязычных постов написали менторы.

In [58]:
texts_list = []
def get_texts(series):
    texts = re.findall(r"'(.+?)',\s*\d+\b|\['(.+?)'", series)
    for text in range(len(texts)):
        texts_list.append(texts[text][1])


data.loc[(data['language'] == 'RUSSIAN') | (data['language'] == 'RUSSIAN AND ENGLISH'), 'posts'].apply(get_texts);

In [59]:
print(f'Всего текстов {len(texts_list)}')

Всего текстов 219


In [60]:
processed_texts = []
for text in texts_list:
    processed_texts.append(preprocess_rus_texts(text))

In [61]:
processed_texts[:5]

['минск искать стажировка',
 'созревать поделиться признание заслуга яндекс практикум возможность',
 'ребята появляться свободный время обновление курс открывать различный формат сотрудничество репост рад конференция митапа подкастый интервью статья издание тестировщик преподаватель создатель авторский курс сообщество аудитория человек основный гордость мочь полезный друг друг знать находить писать телегр отдельный пункт прямой эфир канал писать форма регистрация прочий ништяки информация эфир ждать близко осень',
 'отдельно продублируюсейчас искать ревьюер следующий курсы ревьюер курс разработчик ревьюер курс системный аналитик ревьюер курс бизнес аналитик',
 'начинать работа тестировщик поддержка выпускник курс упорно искать работа сразу тестирование сложный почему рассматриваться вакансия служба технический поддержка поддержка иметь несколько уровень просто телефон начинать разбирать быстрый обращение пользователь уровень проводить анализ уровень подразумевать анализ проблема прод с

In [62]:
processed_texts = [i.split() for i in processed_texts if i != '']

В результате мы получили списки слов из каждого поста.

In [63]:
processed_texts[2]

['ребята',
 'появляться',
 'свободный',
 'время',
 'обновление',
 'курс',
 'открывать',
 'различный',
 'формат',
 'сотрудничество',
 'репост',
 'рад',
 'конференция',
 'митапа',
 'подкастый',
 'интервью',
 'статья',
 'издание',
 'тестировщик',
 'преподаватель',
 'создатель',
 'авторский',
 'курс',
 'сообщество',
 'аудитория',
 'человек',
 'основный',
 'гордость',
 'мочь',
 'полезный',
 'друг',
 'друг',
 'знать',
 'находить',
 'писать',
 'телегр',
 'отдельный',
 'пункт',
 'прямой',
 'эфир',
 'канал',
 'писать',
 'форма',
 'регистрация',
 'прочий',
 'ништяки',
 'информация',
 'эфир',
 'ждать',
 'близко',
 'осень']

In [64]:
print(f'Опубликовано русскоязычных текстов: {len(processed_texts)}')

Опубликовано русскоязычных текстов: 177


Всего было опубликовано 177 русскоязычных постов. На основе этих данных мы будем строить модель. На первом шаге воспользуемся методами библиотеки *gensim*, чтобы получить биграмы из слов в текстах.

In [65]:
bigram = gensim.models.Phrases(processed_texts) # higher threshold fewer phrases
bigram_mod = gensim.models.phrases.Phraser(bigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [66]:
# Create bigrams
data_words_bigrams = make_bigrams(processed_texts)

Далее посмотрим на сформированный корпус текстов.

In [67]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)
# Create Corpus
texts = data_words_bigrams
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:2])

[[(0, 1), (1, 1), (2, 1)], [(3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]]


In [68]:
id2word[76]

'доучивать'

In [69]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[10:11]]

[[('проводить', 1),
  ('студент', 1),
  ('день', 1),
  ('знание', 1),
  ('мгу', 1),
  ('навык', 1),
  ('подход', 1),
  ('получение', 1),
  ('предпринимательство', 1),
  ('преподавание', 1),
  ('продуктовый', 1),
  ('тема', 1),
  ('технологичный', 1)]]

Наконец, обучим языковую модель.

In [70]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                            id2word=id2word,
                                            num_topics=10, 
                                            random_state=100,
                                            update_every=20,#как часто параметры модели должны обновляться
                                            chunksize=5, #количество документов, которые будут использоваться в каждом обучающем чанке
                                            passes=100, #общее количество проходов обучения
                                            alpha='symmetric', #влияет на разреженность тем
                                            per_word_topics=True)

Визуализируем наши темы. Нам удалось не только добиться максимальной разреженности темы, но и получить содержательные и интерпретируемые темы.

In [72]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word) 
vis

/Users/anshilina/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.005283 -0.130982       1        1  20.619116
0      0.063431 -0.019881       2        1  16.221885
2      0.171783 -0.025999       3        1  13.986624
9     -0.115472 -0.021627       4        1  13.571981
7      0.083361  0.042348       5        1   9.325852
6     -0.062944 -0.114497       6        1   8.041266
4     -0.048095  0.108441       7        1   7.232440
8      0.000768  0.097376       8        1   4.579993
3     -0.020562  0.027651       9        1   3.786978
5     -0.066987  0.037169      10        1   2.633865, topic_info=            Term       Freq      Total Category  logprob  loglift
273         план  19.000000  19.000000  Default  30.0000  30.0000
301         цель  22.000000  22.000000  Default  29.0000  29.0000
278    продавать  18.000000  18.000000  Default  28.0000  28.0000
468       задача  23.000000  23.000000  Default  27.0000  27.0000
233         тема  19.000000  19.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
194         день   0.902997  14.059890  Topic10  -5.4561   0.8914
624       крутой   0.873644   6.542553  Topic10  -5.4892   1.6233
222         язык   0.848606   9.706812  Topic10  -5.5183   1.1997
995       далеко   0.818734   9.309415  Topic10  -5.5541   1.2057
990  иностранный   0.818729   3.690152  Topic10  -5.5541   2.1310

[607 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
2576      2  0.704856  автоматически
235       5  0.468818   адекватность
980       6  0.645258      айтишечка
1136      7  0.494592          айфон
1999      3  0.842454     актуальный
...     ...       ...            ...
222       1  0.412082           язык
222       3  0.103020           язык
222       4  0.412082           язык
222      10  0.103020           язык
9         6  0.645269         яндекс

[1059 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3, 10, 8, 7, 5, 9, 4, 6])

#### <a id='toc1_1_3_3_'></a>[Топ-10 тем по ключевым словам (рус)](#toc0_)

Темы расположены по частоте встречаемости в публикациях.

**Тема 1: истории о работе ментором**

*Ключевые слова: ментор, работа, менторинг, поддержка, проходить, помогать, коучинг, находить, понимать, работать, а также цель, курс, рассказывать*

В публикациях на эту тему наставники рассказывают о своей профессии: цели, проекты, курсы, а также о коучинге и поддержке начинающих специалистов.

**Тема 2: истории о работе в сфере аналитики данных, менторы делятся своим опытом работы**

*Ключевые слова: бизнес, компания, клиент, проект, сессия, хороший, продукт, рынок, интернет, датасет, модель, а также возможность, книга, понимание*

Многие наставники в первую очередь сами являются дата саентистами и аналитиками данных, поэтому многие их публикации посвящены рассказам о своей основной деятельности.

**Тема 3: поиск работы в сфере дата саенс**

*Ключевые слова: рынок, вакансия, сайт поиск, машинное обучение, встреча, навык, коуч, учиться, конверсия, граница (поиск работы за границей) и др.*

Также наставники рассказывают о поиске работы (в т.ч. за границей), делятся инсайдами рынка трудоустройства, делятся новыми вакансиями, анонсами тематических встреч.

**Тема 4: навыки, востребованные у работодателей**

*Ключевые слова: опыт, команда, модель, знание, тестирование, компания, система, бизнес, собес (собеседование), опыт работы, нужный, а также развитие, инструмент, использовать, мочь, гайд, язык*

Еще одна тема, связанная трудоустройством – это навыки, которые по мнению работадателей необходимы для соискателя на позицию в сфере дата саенс. Другими словами, какие навыки необходимо прокачать тем, кто хочет трудоустроиться в этой сфере.

**Тема 5: перспективы экономического роста и развития в сфере дата саенс**

*Ключевые слова: рост, реклама, продавать, бизнес, предприниматель, человек, скорость, косвенный конкурент, сверхзадача, изменение, а также рубль, млн., делать, деньги*

Наставники также обсуждают перспективы IT-специалистов, проектов, в т.ч. стартапы, предпринимателей – эта тема связана с бизнесом и экономикой.

**Тема 6: рассказы о задачах наставников и менторов**

*Ключевые слова: групповой менторинг, вебинар, карьерный, коуч, друг, партнер, команда, опытный, разработка, мастер, научиться, продолжать, а также специфические: Практикум, Москва*

Не спроста данная тема на графике расположена близко к **Теме 1** – они близки по содержанию. Но в отличае от темы 1, где фокус повествования смещен на описание профессии в целом, в **Теме 6** менторы рассказывают о задачах, которые перед ними стоят: групповой менторинг, проведение вебинаров и карьерных консультаций, работа в команде, обучение, поддержка, ментор – это друг для своих подопечных. В теме также часто упоминался Практикум – скорее всего менторы из Яндекс Практикума часто публикуют посты на эту тему.

**Тема 7: полезные советы от менторов**

*Ключевые слова: полезно, качество, каждый, профессия, требование, курс, аналитика, практически, осознавать, история, расширение, круг, прочитывать*

Менторы также в публикациями часто делаться полезными советами, ссылками на курсы, книги, полезные для расширения кругозора и погружения в профессию аналитика или дата саентиста.

**Тема 8: важность стратегического планирования, как разработать стратегию своего карьерного развития**

*Ключевые слова: план, цель, работать, регулярный, горизонт (планирования), движение, двигаться, планирование, планировать, копилочка, стратегический, достижение, подводить, итог, завтра, ограничение, месяц* 

В качестве отдельного тематического блока можно выделить стратегическое планирование: менторы часто пишут о стратегиях карьерного развития, подчеркивают необходимость ставить чёткие цели и строить планы, двигаться на пути их достижения.

**Тема 9: рассказы о методиках обучения, в т. ч. от коллег**

*Ключевые слова: методика, учет, развитие, решение, эксперт, обучение, цифра, стоить искать, анализ, проводить, коллега*

Темы 9 и 10 наименее популярные (их меньше всего поднимают в своих публикациях менторы). **Тема 9** скорее посвещена методикам менторинга и наставничества, скорее всего аудитория таких постов – коллеги-менторы. 

**Тема 10: образование, освоение навыков в сфере анализа данных, анонсы и приглашения на вебинары**

*Ключевые слова: аналитика, тема, данные, практика, мониторинг, приветствовать, вебинар, ревьюер, оценка, курс, школа, навык, быстро, а также работа, любить*

Исходя из ключевых слов в **Теме 10**, можно предположить, что эти публикации направлены на анонсирование вебинаров, курсов, школ и матерских, возможность получить практических опыт в сфере аналитики, по работе с данными.

#### <a id='toc1_1_3_4_'></a>[Модель на англоязчных текстах](#toc0_)

In [73]:
data.head(10)

,name,works_at,profile_url,posts,language
0,Елена Хасанова,"QA Head, QA Team-lead, QA-Lead, Наставник Янде...",https://www.linkedin.com/in/yelena-khassanova/,"[['Минск, кто ищет стажировки QA - для вас 🙂 '...",RUSSIAN
1,Алексей Становкин,Ведущий Software QA Engineer,https://www.linkedin.com/in/alex7cri/,"[['https://lnkd.in/gx-UipHF#нетУбийствам'], ['...",ENGLISH
2,Дмитрий Липин,СТО в Yandex Platform Engineering,https://www.linkedin.com/in/dmitrylipin/,[['Очередной митап команды Yandex Infrastructu...,ENGLISH
3,Ramil Gataullin,Data Engineer at Toptal,https://www.linkedin.com/in/yaugear/,[],None
4,Александр Романов,CTO (CIO) at iRidi | Создаю и трансформирую си...,https://www.linkedin.com/in/romanovalexander/,[['Два дня преподавания в МГУ )))Провел 120 ст...,RUSSIAN
5,Aleksandr Shabanov,ML engineer,https://www.linkedin.com/in/alshabanov/,[['Hi everyone - I am looking for a new role a...,ENGLISH
6,Pavel Kiryanov,QA Lead,https://www.linkedin.com/in/pavel-kiryanov-974...,"[['Я вот человек простой, чет написал и коммен...",RUSSIAN
7,Amir Safiullin,Machine Learning Engineer,https://www.linkedin.com/in/safiullinamir/,"[['22 июня, в 19:00, приглашаем на VK Tech Tal...",ENGLISH
8,Soslan Tabuev,datascience.xyz,https://www.linkedin.com/in/soslan-tabuev/,[['An interesting review of Transformer Zoo by...,ENGLISH
9,Alexander Konkin,Business Data Science Team Lead – Exness,https://www.linkedin.com/in/alexander-konkin-0...,[],None


In [74]:
texts_list = []
def get_texts(series):
    texts = re.findall(r"'(.+?)',\s*\d+\b|\['(.+?)'", series)
    for text in range(len(texts)):
        texts_list.append(texts[text][1])

data.loc[data['language'] == 'ENGLISH', 'posts'].apply(get_texts);

In [75]:
print(f'Всего текстов {len(texts_list)}')

Всего текстов 321


In [76]:
processed_texts = []
for text in texts_list:
    processed_texts.append(preprocess_eng_texts(text))

In [77]:
processed_texts = [i.split() for i in processed_texts if i != '']

Публикаций на английском больше: 301 текст.

In [79]:
print(f'Всего опубликовано англоязычных текстов: {len(processed_texts)}')

Всего опубликовано англоязычных текстов: 301


In [80]:
bigram = gensim.models.Phrases(processed_texts) # higher threshold fewer phrases. min_count=5, threshold=10
bigram_mod = gensim.models.phrases.Phraser(bigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [81]:
# Create bigrams
data_words_bigrams = make_bigrams(processed_texts)

In [82]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)
# Create Corpus
texts = data_words_bigrams
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:2])

[[(0, 1)], [(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]]


In [83]:
id2word[55]

'opportun'

In [84]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[10:11]]

[[('highli_recommend', 1),
  ('anoth', 1),
  ('challeng', 1),
  ('cours', 1),
  ('edx', 1),
  ('excel', 1),
  ('mitx', 1),
  ('veri', 1)]]

In [85]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                            id2word=id2word,
                                            num_topics=10, 
                                            random_state=100,
                                            update_every=5,#как часто параметры модели должны обновляться
                                            chunksize=5, #количество документов, которые будут использоваться в каждом обучающем чанке
                                            passes=80, #общее количество проходов обучения
                                            alpha='asymmetric', #влияет на разреженность тем
                                            per_word_topics=True)

In [86]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word) 
vis

/Users/anshilina/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.021781  0.301437       1        1  25.695271
1     -0.311794 -0.019887       2        1  14.202612
0      0.093419 -0.073589       3        1  12.124046
8      0.090707  0.010806       4        1  10.577086
5      0.065649 -0.115856       5        1   9.527307
2     -0.039442 -0.038138       6        1   9.231576
3     -0.000515 -0.041873       7        1   8.096304
6      0.038171 -0.016422       8        1   5.442414
4      0.023538 -0.008949       9        1   3.748274
7      0.018486  0.002470      10        1   1.355110, topic_info=             Term       Freq      Total Category  logprob  loglift
384   happi_share  50.000000  50.000000  Default  30.0000  30.0000
386     start_new  39.000000  39.000000  Default  29.0000  29.0000
81            use  32.000000  32.000000  Default  28.0000  28.0000
385         posit  35.000000  35.000000  Default  27.0000  27.0000
381          data  33.000000  33.000000  Default  26.0000  26.0000
...           ...        ...        ...      ...      ...      ...
146        recent   0.411402   9.274231  Topic10  -5.3165   1.1859
377    makushenko   0.028237   0.681811  Topic10  -7.9954   1.1172
378          olga   0.028237   0.681811  Topic10  -7.9954   1.1172
140        market   0.416568  13.945419  Topic10  -5.3040   0.7904
1909         anar   0.028317   3.039865  Topic10  -7.9925  -0.3748

[371 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1919      9  0.770269      aca
1504      4  0.866762  academi
477       6  0.924455  acceler
373       8  0.877042   achiev
1483     10  0.926908     adob
...     ...       ...      ...
1561      6  0.881691     xahi
1838      1  0.988345  xgboost
1984      9  0.521599   yapmak
132       4  0.958825     year
2226      5  0.925953     yeni

[351 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 2, 1, 9, 6, 3, 4, 7, 5, 8])

#### <a id='toc1_1_3_5_'></a>[Топ-10 тем по ключевым словам (англ)](#toc0_)

Темы расположены по частоте встречаемости в публикациях.

**Тема 1: Рассказы о проектах**

*Ключевые слова:* work, analytics, data science, project, xgboost, chatgpt, help, machin learning, intern, team, research

Рассказы о своих исследовательских проектах в области анализа данных и data science, также упоминание инструментов машинного обучения (xgboost).

**Тема 2: Личный профессиональный рост**

*Ключевые слова:* start new position, data, analyst, product, profi, train, teach, data scientist, growth, junior position

Истории о личном профессиональном росте, начале своей карьеры.

**Тема 3: Финтех сфера**

*Ключевые слова:* fintech, crypto, conference, participation, market, challenge, future, model, program, use

Публикации, связанные с финтех сферой, криптовалюте, рынках, участиях в конференциях

**Тема 4: Возможности обучения**

*Ключевые слова:* learn, develop, experience, knowledge, academic, sql,  course, finish, education, team, algorithm, data scientist

Рассказы о возможностях обучения, развития навыков, получения опыта в сфере data science. Также упоминаются отдельные курсы, возможности по изучению SQL.

**Тема 5:** –

*Ключевые слова:* - 

В этой теме алгоритм собрал слова, которые не относятся к английскому языку. Видимо, некоторые авторы писали не только на английском, либо дублировали свои текты на других языках. 

**Тема 6: Полезные ресурсы**

*Ключевые слова:* people analysts, webinar, important, insight, core, talent, leader, acceleration, global, resource 

Рассказы о вебинарах, возможностях по акселерации, о талантривых людях в сфере аналитики. Делятся важными ресурсами, своими инсайтами и проч.

**Тема 7: Возможности повышения квалификации**

*Ключевые слова:* python, engine, certificate, statistics, obtain new, happy share, data science, machine learning, platform, language, vision, github, repro, senior

Делятся информацией о возможностях повысить квалификацию, получить новые навыки и сертификаты в сфере data science и machine learning. Также по-видимому делятся платформами для изучения языков програмирования.

**Тема 8: Крупные игроки на рынке**

*Ключевые слова:* Microsoft, Google, Amazon, provide, achieved, forecast, market, uncertainty, pandemic, usd, million 

Тема посвящена обсуждению крупных игроков на рынке IT (они же работадатели) Microsoft, Google, Amazon, а также рынку, нестабильности на рынке. Делятся своими прогнозами.

**Тема 9-10: Поиск работы**

*Ключевые слова:* opportunity, connect, open to work, opportunity offer, everyone look, new offer, advanced analytics, frontend

Темы 9 и 10 модель не смогла разделить. В этих темах собраны публикации, связанные с поиском работы, вакансий в сфере data science и data analytics.


### <a id='toc1_1_4_'></a>[Подготовка данных для языковой модели по самым популярным постам](#toc0_)

Популярность постов определялась по количеству реакций.

In [87]:
data_russian.head()

,name,works_at,profile_url,posts,language,posts_processed,words
0,Елена Хасанова,"QA Head, QA Team-lead, QA-Lead, Наставник Янде...",https://www.linkedin.com/in/yelena-khassanova/,"[['Минск, кто ищет стажировки QA - для вас 🙂 '...",RUSSIAN,минск искать стажировка созревать поделиться п...,"[минск, искать, стажировка, созревать, поделит..."
1,Александр Романов,CTO (CIO) at iRidi | Создаю и трансформирую си...,https://www.linkedin.com/in/romanovalexander/,[['Два дня преподавания в МГУ )))Провел 120 ст...,RUSSIAN,день преподавание мгу проводить студент получе...,"[день, преподавание, мгу, проводить, студент, ..."
2,Pavel Kiryanov,QA Lead,https://www.linkedin.com/in/pavel-kiryanov-974...,"[['Я вот человек простой, чет написал и коммен...",RUSSIAN,человек простой чета написать коммент читаюно ...,"[человек, простой, чета, написать, коммент, чи..."
3,Александр Шобухов,CrowdFunding - Народное финансирование.,https://www.linkedin.com/in/%D0%B0%D0%BB%D0%B5...,[['Здравствуй Друг! Посмотри новый проект! Вы ...,RUSSIAN,друг посмотреть новый проект человек предприим...,"[друг, посмотреть, новый, проект, человек, пре..."
4,Aleksandr Romanov,CTO (CIO) at iRidium mobile | I Create and Tra...,https://www.linkedin.com/in/romanovalexander/,[['Два дня преподавания в МГУ )))Провел 120 ст...,RUSSIAN,день преподавание мгу проводить студент получе...,"[день, преподавание, мгу, проводить, студент, ..."


Для того, чтобы получить топ-10 постов по количеству реакций воспользуемся простыми регулярными выражениями.

In [88]:
post_to_sort_rus = []

def get_best_posts(series):
    texts = series.split('], [')
    for text in texts:
        text_react = re.findall(r"'(.+?)'", text)
        try:
            text_low = text_react[0].lower()
            text_proc = ' '.join(re.sub(r'[^а-яё ]', ' ', text_low).split())
            if len(text_proc) > 1:
                post_to_sort_rus.append([text_proc, int(text_react[1])])
        except:
            continue

data_russian['posts'].apply(get_best_posts);

Изучим самые популярные публикации: анонсы конкурсов, приглашение людей на наставничество (помощь новичкам), рассказы про опыт работы наставником, истории про работу в сфере анализа данных, полезные советы и полезные ресурсы, поиск работы. Все эти посты иллюстрируют многие из русскоязычных тем, которые мы выявили в ходе моделирования.

In [89]:
sorted(post_to_sort_rus, key=lambda x: x[1], reverse = True)[:10]

[['новогодние подарки от для специалистов стать сертифицированным бесплатно что бы получать всю информацию раньше других подписывайся на мой телеграмм лучшая благодарность лайк коммент репост что бы получать всю информацию раньше других подписывайся на мой телеграмм польша релокация',
  917],
 ['друзья приветствую я готов взять себе еще несколько человек на наставничество по наставничество по простому это когда более опытный человек помогает менее опытному пройти тот же путь который он уже прошел с каким запросами я работаю в основном я хочу сменить профессию и стать я хочу стать я хочу стать я хочу стать я хочу получать в раза больше денегкак это все проходит сначала мы созваниваемся и знакомимся обсуждаем твой запрос и твои желания составляем план движения к цели сюда может входить технический подтягивание пробелов в теории и практике подготовка подготовка к собеседованиям один раз в неделю мы созваниваемся на час и двигаемся к твоей целичто ты получаешь персонального ментора наставн

Всего 102 из 117 постов были отмечены получили реакциями. Для построения языковой модели по самым популярным постам возьмем топ-50 постов по количеству реакций.

In [90]:
sorted_posts_rus = sorted(post_to_sort_rus, key=lambda x: x[1], reverse = True)
sorted_posts_rus = [i[0] for i in sorted_posts_rus]
print(f'Кол-во публикаций с реакциями: {len(sorted_posts_rus)}')
sorted_posts_rus = sorted_posts_rus[:51]

Кол-во публикаций с реакциями: 102


Теперь подготовим топ-50 англоязычных постов.

In [91]:
data_english.head()

,name,works_at,profile_url,posts,language,posts_processed,words
0,Алексей Становкин,Ведущий Software QA Engineer,https://www.linkedin.com/in/alex7cri/,"[['https://lnkd.in/gx-UipHF#нетУбийствам'], ['...",ENGLISH,uiphf engin interview without look resum crede...,"[uiphf, engin, interview, without, look, resum..."
1,Дмитрий Липин,СТО в Yandex Platform Engineering,https://www.linkedin.com/in/dmitrylipin/,[['Очередной митап команды Yandex Infrastructu...,ENGLISH,yandex infrastructur backend devop sre open so...,"[yandex, infrastructur, backend, devop, sre, o..."
2,Aleksandr Shabanov,ML engineer,https://www.linkedin.com/in/alshabanov/,[['Hi everyone - I am looking for a new role a...,ENGLISH,everyon look new role would appreci support th...,"[everyon, look, new, role, would, appreci, sup..."
3,Amir Safiullin,Machine Learning Engineer,https://www.linkedin.com/in/safiullinamir/,"[['22 июня, в 19:00, приглашаем на VK Tech Tal...",ENGLISH,tech talk coheua implicit feedback need vktech...,"[tech, talk, coheua, implicit, feedback, need,..."
4,Soslan Tabuev,datascience.xyz,https://www.linkedin.com/in/soslan-tabuev/,[['An interesting review of Transformer Zoo by...,ENGLISH,interest review transform zoo grigori sapunov ...,"[interest, review, transform, zoo, grigori, sa..."


In [92]:
post_to_sort_eng = []

def get_best_posts(series):
    texts = series.split('], [')
    for text in texts:
        text_react = re.findall(r"'(.+?)'", text)
        try:
            text_low = text_react[0].lower()
            text_proc = ' '.join(re.sub(r'[^a-z ]', ' ', text_low).split())
            if len(text_proc) > 1:
                post_to_sort_eng.append([text_proc, int(text_react[1])])
        except:
            continue

data_english['posts'].apply(get_best_posts);

In [93]:
sorted(post_to_sort_eng, key=lambda x: x[1], reverse = True)[1:11] # first one not english language

[['vacancy cv job ps', 520],
 ['monai the medical open network for ai is continuing to expand its capabilities to help developers accelerate their medical imaging ai workflows the release of monai brings exciting updates including monai model zoo auto d segmentation and active learning in monai label https nvda ws s naql',
  407],
 ['s the news https lnkd in dkhyxsmbthis must be great for people like me who struggled to get past this work permit wall when applying for a job if there are swiss recruiters in my network who are looking for someone in machine learning or neurotech send me a message fyi i hold a phd degree from swiss university epfl which should make things even easier i would also highly appreciate if anyone could refer me this whole cold approach of applying through linkedin doesn',
  399],
 ['xa xa xa xa we want to hire a chief data officer xa xa xa xa xa xa xa xa xa xa xa xa ok great xa xa xa xa xa yeah we would like to be more data driven xa xa xa xa xa xa xa xa xa xa 

In [94]:
sorted_posts_eng = sorted(post_to_sort_eng, key=lambda x: x[1], reverse = True)
sorted_posts_eng = [i[0] for i in sorted_posts_eng]
print(f'Кол-во публикаций с реакциями: {len(sorted_posts_eng)}')
sorted_posts_eng = sorted_posts_eng[:51]

Кол-во публикаций с реакциями: 250


### <a id='toc1_1_5_'></a>[Построение языковой модели по топ-50 популярным постам](#toc0_)

Построим модель по русскоязычным текстам.

In [95]:
processed_texts_rus = []
for text in sorted_posts_rus:
    processed_texts_rus.append(preprocess_rus_texts(text))

In [96]:
processed_texts_rus = [i.split() for i in processed_texts_rus if i != '']
print(len(processed_texts_rus))

51


In [97]:
bigram = gensim.models.Phrases(processed_texts_rus)
bigram_mod = gensim.models.phrases.Phraser(bigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [98]:
# Create bigrams
data_words_bigrams = make_bigrams(processed_texts_rus)

In [99]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)
# Create Corpus
texts = data_words_bigrams
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:2])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 2), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1)], [(8, 2), (15, 4), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 3), (43, 1), (44, 2), (45, 1), (46, 2), (47, 2), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 2), (58, 3), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 2), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 5), (72, 1), (73, 3), (74, 1), (75, 2)]]


In [100]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[10:11]]

[[('информация', 1),
  ('подписываться', 1),
  ('получать', 2),
  ('становиться', 1),
  ('хороший', 2),
  ('готовый', 2),
  ('друг', 1),
  ('мочь', 1),
  ('обсуждать', 1),
  ('подготовка', 1),
  ('помогать', 1),
  ('работать', 1),
  ('составлять', 1),
  ('хотеть', 1),
  ('человек', 2),
  ('внутри', 1),
  ('достигать', 1),
  ('именно', 1),
  ('решать', 1),
  ('сделать', 2),
  ('ссылка', 1),
  ('канал', 1),
  ('комментарий', 1),
  ('полезный', 1),
  ('нужно', 1),
  ('заранее', 1),
  ('предложение', 1),
  ('собирать', 1),
  ('деньги', 1),
  ('поэтому', 1),
  ('актуальный', 1),
  ('алгоритм', 1),
  ('важный', 1),
  ('вилка', 1),
  ('волшебный', 1),
  ('встреча', 1),
  ('выписывать', 1),
  ('глаз', 1),
  ('договоренность', 1),
  ('заинтересованный', 1),
  ('итак', 1),
  ('итог', 1),
  ('компромисс', 1),
  ('конец', 1),
  ('краткий', 1),
  ('максимум', 1),
  ('напряжение', 1),
  ('настраиваться', 1),
  ('ничто', 1),
  ('образ', 1),
  ('общение', 1),
  ('оказываться', 1),
  ('определять', 1),

In [101]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                            id2word=id2word,
                                            num_topics=5, 
                                            random_state=100,
                                            update_every=5,#как часто параметры модели должны обновляться
                                            chunksize=5, #количество документов, которые будут использоваться в каждом обучающем чанке
                                            passes=100, #общее количество проходов обучения
                                            alpha='auto', #влияет на разреженность тем
                                            per_word_topics=True)

In [102]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word) 
vis

/Users/anshilina/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.061014  0.118650       1        1  32.818203
3      0.081633 -0.003284       2        1  27.245055
4      0.057029 -0.104019       3        1  19.007695
0      0.039879 -0.013446       4        1  11.640665
1     -0.239554  0.002099       5        1   9.288383, topic_info=         Term       Freq      Total Category  logprob  loglift
142    проект   7.000000   7.000000  Default  30.0000  30.0000
151   решение   8.000000   8.000000  Default  29.0000  29.0000
75    человек  16.000000  16.000000  Default  28.0000  28.0000
179  вакансия  15.000000  15.000000  Default  27.0000  27.0000
123  начинать   8.000000   8.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
79     бизнес   0.877523   5.583354   Topic5  -5.4835   0.5260
112   команда   0.710708   4.695077   Topic5  -5.6944   0.4884
113  компания   0.702344  12.822660   Topic5  -5.7062  -0.5281
121    модель   0.613340   5.298130   Topic5  -5.8417   0.2202
144    работа   0.553052  15.563896   Topic5  -5.9452  -0.9609

[256 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
176       1  0.969066  агрегатор
596       1  0.913223  айтишечка
282       2  0.741101   актуарий
1083      5  0.860538     анализ
1084      5  0.860538     апатия
...     ...       ...        ...
75        1  0.238578    человек
75        2  0.357866    человек
75        3  0.059644    человек
75        5  0.298222    человек
724       3  0.800975     чтение

[315 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 5, 1, 2])

#### <a id='toc1_1_5_1_'></a>[Топ-5 популярных тем по реакциям (рус)](#toc0_)

Темы расположены по частоте встречаемости в публикациях.

**Тема 1: Поиск работы**

*Ключевые слова:* вакансия, работа, опыт, компания, подходить, тестирование, опыт работа, искать, новый, сайт поиск, поиск работа, разработчик

Публикации, связанные с поиском работы, открытыми вакансиями.

**Тема 2: Приглашение на групповой менторинг**

*Ключевые слова:* групповой менторинг, ментор, менторинг, коучинг, коуч, получать, становиться, научиться, результат, профессия

Приглашение на групповой менторинг, рассказы о результатах работы с менторами и коучами.

**Тема 3: Планирование, постановка целей**

*Ключевые слова:* план, цель, курс, планирование, горизонт, книга, регулярный, собеседование, групповой менторинг, вместе

Полезные советы, планирование в рамках программ группового менторинга. Но акцент смещен на стратегию работы в рамках курсов менторов (в **Теме 2** основной фокус на приглашение в курсы).

**Тема 4: Задачи менторов**

*Ключевые слова:* сессия, навык, делать, коучинг, хотеть, менторинг, развивать, понимание, помогать, урок, учиться, создавать

Рассказы о задачах менторов, какие навыки развивают, с чем помогают.

**Тема 5: Мотивирующие посты**

*Ключевые слова:* проект, решение, принимать, анализ, вовлекать, заниматься, браться, находить, апатия, страстный, полезно

Исходя из ключевых слов, тексты публикаций направлены на мотивацию своей аудитории делать, заниматься, браться, находить, принимать решения.

Теперь построим модель по русскоязычным текстам.

In [103]:
processed_texts_eng = []
for text in sorted_posts_eng:
    processed_texts_eng.append(preprocess_eng_texts(text))

In [104]:
processed_texts_eng = [i.split() for i in processed_texts_eng if i != '']
print(len(processed_texts_eng))

51


In [105]:
bigram = gensim.models.Phrases(processed_texts_eng)
bigram_mod = gensim.models.phrases.Phraser(bigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [106]:
# Create bigrams
data_words_bigrams = make_bigrams(processed_texts_eng)

In [107]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)
# Create Corpus
texts = data_words_bigrams
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:2])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)], [(22, 1), (23, 1)]]


In [108]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[10:11]]

[[('excit', 1),
  ('learn', 1),
  ('thi', 1),
  ('work', 1),
  ('grow', 1),
  ('project', 1),
  ('current', 1),
  ('depart', 1),
  ('differ', 1),
  ('foreign', 1),
  ('ongo', 1),
  ('relat', 1),
  ('role', 1),
  ('specialist', 1)]]

In [109]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                            id2word=id2word,
                                            num_topics=5, 
                                            random_state=100,
                                            update_every=10,#как часто параметры модели должны обновляться
                                            chunksize=5, #количество документов, которые будут использоваться в каждом обучающем чанке
                                            passes=100, #общее количество проходов обучения
                                            alpha='auto', #влияет на разреженность тем
                                            per_word_topics=True)

In [110]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word) 
vis

/Users/anshilina/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.173702  0.001355       1        1  47.546797
1      0.052407  0.072583       2        1  18.545765
3      0.051157 -0.057997       3        1  18.264642
2      0.028309 -0.004827       4        1   8.121636
0      0.041829 -0.011114       5        1   7.521160, topic_info=               Term      Freq      Total Category  logprob  loglift
243       start_new  30.00000  30.000000  Default  30.0000  30.0000
241     happi_share  32.00000  32.000000  Default  29.0000  29.0000
242           posit  28.00000  28.000000  Default  28.0000  28.0000
234           engin  27.00000  27.000000  Default  27.0000  27.0000
408          comput  25.00000  25.000000  Default  26.0000  26.0000
..              ...       ...        ...      ...      ...      ...
168             eld   0.32405   0.957812   Topic5  -5.7118   1.5037
169           etmek   0.32405   0.957812   Topic5  -5.7118   1.5037
170           eyden   0.32405   0.957812   Topic5  -5.7118   1.5037
171             eyi   0.32405   0.957812   Topic5  -5.7118   1.5037
172  eylemlerinizin   0.32405   0.957812   Topic5  -5.7118   1.5037

[233 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
490       1  0.965490  academi
24        2  0.634999  acceler
402       2  0.634999    advic
261       1  0.793911  analyst
124       1  0.443761   analyt
...     ...       ...      ...
92        1  0.345588    would
92        2  0.345588    would
287       3  0.637997      xsl
223       5  0.842377   yapmak
117       1  0.965490     yeah

[196 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 4, 3, 1])

#### <a id='toc1_1_5_2_'></a>[Топ-5 популярных тем по реакциям (англ)](#toc0_)

Темы расположены по частоте встречаемости в публикациях.

**Тема 1: Сообщения о начале новой позиции**

*Ключевые слова:* happy share, start new position, computer research, data science, analyst, mashine learning

**Тема 2: Истории о своих проектах**

*Ключевые слова:* medical, software, learn, appreciate, solution, new, look, printer

**Тема 3: Компании и стратегии развития в сфере data science**

*Ключевые слова:* data science, company, new, analytics, data, member, strategy, technology

**Тема 4: Проекты и стажировки в сфере data science**

*Ключевые слова:* team, project, internship, provide, make, insight, searchchat, chatgpd

**Тема 5:** – 

*Ключевые слова:* –

В этой теме алгоритм собрал слова, которые не относятся к английскому языку. Видимо, некоторые авторы писали не только на английском, либо дублировали свои текты на других языках.